In [1]:
import pandas as pd
import numpy as np

In [2]:
# import enrollment and proficiency data
df_e = pd.read_csv('data/finalized/enrollment.csv')
df_p = pd.read_csv('data/finalized/proficiency.csv')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (13,15,16,22,23,24,25,26,30,31,32,33,34,35,38,39,47,48,50,51,54,56,57,59,60,64,65,66,67,68,69,72,73,74,75,77,93,94,95,96,97,98) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (66,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_e['grade'] = df_e['grade'].astype(str)
df_p['grade'] = df_p['grade'].astype(str)

In [4]:
# merge enrollment and proficiency data
df = df_p.merge(df_e, on=['year', 'state', 'district_id', 'school_id', 'grade', 'group'], how='outer', suffixes=('_p', '_e'))

In [5]:
df[df.state.eq('ca')].to_csv('temp.csv')

In [6]:
print(df.state.drop_duplicates())

0    ma
Name: state, dtype: object


In [7]:
# calculate rows in join
print(df.shape[0])
df_left = df[df.pct_at_level.isnull()].groupby(['state']).size().to_frame(name = 'left').reset_index()
df_inner = df[~df.num.isnull() & ~df.pct_at_level.isnull()].groupby(['state']).size().to_frame(name = 'inner').reset_index()
df_right = df[df.num.isnull()].groupby(['state']).size().to_frame(name = 'right').reset_index()
print(df[df.num.isnull()].shape[0] + df[~df.num.isnull() & ~df.pct_at_level.isnull()].shape[0] + df[df.pct_at_level.isnull()].shape[0])

171431
171431


In [8]:
df_counts = df_left.merge(df_inner, on='state', how='outer').merge(df_right, on='state', how='outer')
print(df_counts)

  state   left  inner  right
0    ma  82613  63074  25744


In [9]:
df_counts['left_%'] = df_counts['left'] / (df_counts['inner'] + df_counts['right'] + df_counts['left'])
df_counts['inner_%'] = df_counts['inner'] / (df_counts['inner'] + df_counts['right'] + df_counts['left'])
df_counts['right_%'] = df_counts['right'] / (df_counts['inner'] + df_counts['right'] + df_counts['left'])

In [10]:
print(df_counts)

  state   left  inner  right    left_%   inner_%   right_%
0    ma  82613  63074  25744  0.481902  0.367926  0.150171
